In [11]:
import numpy as np
import operator
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, LSTM, SpatialDropout1D, Dropout, Flatten
#from keras.datasets import imdb
import pandas as pd
import numpy as np
#устанавливаем seed
np.random.seed(42)
import pymorphy2
import re
import datetime
import sklearn.feature_extraction.text as fex
morph = pymorphy2.MorphAnalyzer()

import tomita
#from tparser import tomita
from tomita_parser import text_parse
import pandas as pd
def normalize(text):
    return ' '.join([morph.parse(str(w))[0].normal_form for w in text.split()])
def clean_text(text):
    text = str(text)
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-\'>><`', ' ', text) #deleting symbols  
    text = re.sub('-', ' ', text) #deleting symbols  
    #text = ' '.join(word[:] for word in text.split() if len(word)>3)
    #text = text.encode("utf-8")
    return text 


In [18]:
#df_train = pd.read_csv('dataset1.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)
df_train = pd.read_csv('datasetNormFinal.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)

df_train = df_train[(df_train[tovarIdColName] != 30025022) & (df_train[tovarIdColName] != 30025023 )]


In [19]:
textColName = "TEXT"
negatColName =  "DRAWBACKS"
positColName = "BENEFITS"
ratingColName = "RATING"
catColName = "CATEGORY_ID"
brandColName = "BRAND_ID"
catNameColName= "CATEGORY_NAME"
tovarIdColName = "PRODUCT"
fTextColName = 'fullText'
fTextColNameNorm = 'fullTextNorm'
tomitaColName = 'tomitaCol'
normPosColName = "NormPos"
normNegColName = "NormNeg"
normTextColName = 'NormCpom'
#df_train.head()

In [20]:
df_test = pd.read_csv('newProd.csv', delimiter=',', encoding='utf-8', error_bad_lines=False)
colList = pd.unique(df_test[catNameColName])


In [36]:
maxFeatures = 10
def printForCat(df_train, df_test,cat):
    category_data = df_train[df_train[catNameColName].str.contains(cat)==True]
    category_data[fTextColName] = category_data[textColName].astype('str') + " "+category_data[positColName].astype('str')
    category_data[fTextColName] = category_data[fTextColName]  +" "+category_data[negatColName].astype('str')

    if (normTextColName in df_train.columns):
        print ("data already normalized")
        category_data[fTextColNameNorm] = category_data[normTextColName].astype('str') + " "+category_data[normNegColName].astype('str')
        category_data[fTextColNameNorm] = category_data[fTextColNameNorm]  +" "+category_data[normPosColName].astype('str')
    else:
        print("start normalize at" + str (datetime.datetime.now()))
        category_data[fTextColNameNorm] = category_data[fTextColName].apply(clean_text).apply(normalize)
        print("finishaed  collect normalize at " + str (datetime.datetime.now().time()))
    category_data_test = df_test[df_test[catNameColName].str.contains(cat)==True]
    category_data_test[fTextColName] = category_data_test[textColName].astype('str') + " "+category_data_test[positColName].astype('str')
    category_data_test[fTextColName] = category_data_test[fTextColName]  +" "+category_data_test[negatColName].astype('str')
    category_data_test[fTextColNameNorm]=category_data_test[fTextColName].apply(clean_text).apply(normalize)
    textAndProdGrS_test = category_data_test.groupby(tovarIdColName )[tovarIdColName, fTextColName].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm_test =  category_data_test.groupby(tovarIdColName )[tovarIdColName, fTextColNameNorm].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrS = category_data.groupby(tovarIdColName )[tovarIdColName, fTextColName].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm =  category_data.groupby(tovarIdColName )[tovarIdColName, fTextColNameNorm].agg(lambda x: " %s " % ', '.join(x)).reset_index()
    textAndProdGrSNorm_All = pd.concat([textAndProdGrSNorm,textAndProdGrSNorm_test])
    textAndProdGrS_All = pd.concat([textAndProdGrSNorm,textAndProdGrSNorm_test])
   # print("total category items " + str(len(textAndProdGrSNorm_All)))
    tf_list = np.array(textAndProdGrSNorm_All)
   # print(tf_list.shape)
    id_tlist = textAndProdGrSNorm_test[tovarIdColName].tolist()

    indList = [i for i in range(0,len(tf_list)) if tf_list[i][0] in id_tlist]
    textAndProdGrS_test[tomitaColName] = textAndProdGrS_test[:][fTextColName].apply(text_parse)
    tfidf = fex.TfidfVectorizer()
    #print("start tfidf at" + str (datetime.datetime.now().time()))
    tfIdfMatrix = tfidf.fit_transform(tf_list[:,1])
    #print("end tf Idf at " + str (datetime.datetime.now().time()))
    ftNames = tfidf.get_feature_names()
    densM = tfIdfMatrix.todense()
    for ind in indList[:]:
        print("#"*50)
        print("product ind " + str(tf_list[ind][0]))
        col = densM[ind].tolist()[0]
        valD = { ftNames[i]:v for i,v in enumerate(col) if v>0  }
        sorted_topWords = sorted(valD.items(), key=operator.itemgetter(1), reverse=True)
        sorted_topWords = [ s for s in sorted_topWords if len(s[0])>3]
        dict_r = { s[0]:s[1] for s in sorted_topWords}
        tomitaD = textAndProdGrS_test[textAndProdGrS_test[tovarIdColName] == textAndProdGrS_All.iloc[ind][tovarIdColName]][tomitaColName]
        sublist = list([x[::,0] for x in tomitaD.values][0])
        set_ret = dict()
        tomitaPairs = tomitaD.tolist()[0]
        #print("start word rating rebuild at " + str (datetime.datetime.now().time()))
        for ts in tomitaPairs:
            ret =0
            tsn = normalize(ts[0])
            if tsn in dict_r.keys():
                ret+=dict_r[tsn]
            for tsPr in ts[1].split(" "):
                tsPrn = normalize(tsPr)
                #print(tsPrn)
                if tsPrn in dict_r.keys():
                 #   print("yes")
                    ret+=dict_r[tsPrn]
            set_ret[ts[1]]=ret
        #print("end word rating rebuild at " + str (datetime.datetime.now().time()))
        sorted_sent = sorted(set_ret.items(), key=operator.itemgetter(1), reverse=True)
        for feature in sorted_sent[:maxFeatures]:
            print(" особенность: "+str(feature[0])+ " важность "+ str(int(float(feature[1])*100)))
    

In [37]:
cat = colList[0][1:]
print(cat)
printForCat(df_train, df_test,cat)

MARTPHONES                               


/usr/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

data already normalized


/usr/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


##################################################
product ind 30025022
 особенность: ОТЛИЧНОЕ КАЧЕСТВО ДЛЯ ТАКОЙ СТОИМОСТИ ТЕЛЕФОНА МОЩНАЯ БАТАРЕЯ важность 81
 особенность: БАТАРЕЯ КОТОРАЯ ДЕРЖИТ ЗАРЯД НЕСКОЛЬКО ДНЕЙ И ОЧЕНЬ важность 80
 особенность: БАТАРЕЯ ДЕРЖАТЬ ЧАСОВ 55-60 С ВКЛЮЧЕННЫМ ИНТЕРНЕТОМ ХОТЯ Я ПЕРИОДИЧЕСКИ ИГРАЮ важность 71
 особенность: БЫТЬ ТЕЛЕФОН С ЕМКОЙ БАТАРЕЯ важность 69
 особенность: ХОТЕТЬ ПРИОБРЕСТИ НЕДОРОГОЙ АППАРАТ С ХОРОШЕЙ КАМЕРОЙ И МОЩНЫМ ЗАРЯД БАТАРЕИ важность 69
 особенность: ЕЩЕ УЖЕ ТРЕТИЙ ДЕНЬ В НАСТРОЙКАХ НЕ МОГУ НАЙТИ БУДИЛЬНИК НЕПЛОХАЯ БАТАРЕЯ КАМЕРА важность 61
 особенность: БАТАРЕЯ ТЕЛЕФОН СЛАБАЯ важность 60
 особенность: ЭТОТ ЖЕ ЗАРЯЖАЕТСЯ ЗА 2 ЧАСА И ОЧЕНЬ ХОРОШО ДЕРЖИТ ЗАРЯД ОТ 2-Х ДО 4-Х ДНЕЙ БАТАРЕЙКА И СКОРОСТЬ РАБОТЫ важность 56
 особенность: БАТАРЕЯ ТОЖЕ ХВАТАЕТ НА ДОЛГО важность 55
 особенность: БАТАРЕЯ ДЕРЖАТЬ ЗАРЯД НЕСКОЛЬКО важность 55
##################################################
product ind 30025023
 особенность: НЕ ТОРМОЗИТ 2 

In [ ]:
densM = tfIdfMatrix.todense()
densM.shape

In [ ]:
sublist = [x[::, 0] for x in tomitaCol.values]

In [ ]:
slAll=list()
for sl in sublist:
    type(sl)
    slAll.append(list(sl))

In [ ]:
sublistInd = [i for i,n in enumerate(ftNames)  if n.upper() in slAll  ]

In [ ]:
sublistInd